# Other VGG Classes

In [52]:
# class VGG(nn.Module):

#     def __init__(self, features, num_classes=1000, init_weights=True):
#         super(VGG, self).__init__()
#         self.features = features
#         self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        
#         self.classifier = nn.Sequential(
#             nn.Linear(512 * 7 * 7, 4096),
#             nn.ReLU(True),
#             nn.Dropout(),
#             nn.Linear(4096, 4096),
#             nn.ReLU(True),
#             nn.Dropout(),
#             nn.Linear(4096, num_classes),
#         )
        
#         if init_weights:
#             self._initialize_weights()

#     def forward(self, x):
#         x = self.features(x)
#         x = self.avgpool(x)
#         x = torch.flatten(x, 1)
#         x = self.classifier(x)
#         return x

#     def _initialize_weights(self):
#         for m in self.modules():
#             if isinstance(m, nn.Conv2d):
#                 nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
#                 if m.bias is not None:
#                     nn.init.constant_(m.bias, 0)
#             elif isinstance(m, nn.BatchNorm2d):
#                 nn.init.constant_(m.weight, 1)
#                 nn.init.constant_(m.bias, 0)
#             elif isinstance(m, nn.Linear):
#                 nn.init.normal_(m.weight, 0, 0.01)
#                 nn.init.constant_(m.bias, 0)

In [ ]:
# class Vgg2D(torch.nn.Module):

#     def __init__(
#             self,
#             input_size,
#             fmaps=12,
#             downsample_factors=[(2, 2), (2, 2), (2, 2), (2, 2)],
#             output_classes=6):

#         super(Vgg2D, self).__init__()

#         self.input_size = input_size

#         current_fmaps, h, w = tuple(input_size)
#         current_size = (h, w)

#         features = []
#         for i in range(len(downsample_factors)):

#             features += [
#                 torch.nn.Conv2d(
#                     current_fmaps,
#                     fmaps,
#                     kernel_size=3,
#                     padding=1),
#                 torch.nn.BatchNorm2d(fmaps),
#                 torch.nn.ReLU(inplace=True),
#                 torch.nn.Conv2d(
#                     fmaps,
#                     fmaps,
#                     kernel_size=3,
#                     padding=1),
#                 torch.nn.BatchNorm2d(fmaps),
#                 torch.nn.ReLU(inplace=True),
#                 torch.nn.MaxPool2d(downsample_factors[i])
#             ]

#             current_fmaps = fmaps
#             fmaps *= 2

#             size = tuple(
#                 int(c/d)
#                 for c, d in zip(current_size, downsample_factors[i]))
#             check = (
#                 s*d == c
#                 for s, d, c in zip(size, downsample_factors[i], current_size))
#             assert all(check), \
#                 "Can not downsample %s by chosen downsample factor" % \
#                 (current_size,)
#             current_size = size

#         self.features = torch.nn.Sequential(*features)

#         classifier = [
#             torch.nn.Linear(
#                 current_size[0] *
#                 current_size[1] *
#                 current_fmaps,
#                 4096),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Dropout(),
#             torch.nn.Linear(
#                 4096,
#                 4096),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Dropout(),
#             torch.nn.Linear(
#                 4096,
#                 output_classes)
#         ]

#         self.classifier = torch.nn.Sequential(*classifier)
    
#     def forward(self, raw):

#         # add a channel dimension to raw
#         # shape = tuple(raw.shape)
#         # raw = raw.reshape(shape[0], 1, shape[1], shape[2])
        
#         # compute features
#         f = self.features(raw)
#         f = f.view(f.size(0), -1)
        
#         # classify
#         y = self.classifier(f)

#         return y

# Other Training frameworks

In [ ]:
#Training on fake data
def train(model, loss_function):
    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    model.train()
    epoch_loss = 0
    num_batches = 0
    model = model.to(device)
    
    for batch_id, (x, y) in enumerate(loader):
        # move input and target to the active device (either cpu or gpu)
        # x = np.stack(x, axis= 0)
        #x, y = x.to(device), y.to(device)
        x = x[np.newaxis]
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        print(x.shape)
        print(y.dtype)
        x, y = x.to(device), y.to(device)
        # zero the gradients for this iteration
        optimizer.zero_grad()

        # apply model and calculate loss
        prediction = model(x)
        loss = loss_function(prediction, y)

        # backpropagate the loss and adjust the parameters
        loss.backward()
        optimizer.step()

        # # log to console
        # if batch_id % log_interval == 0:
        #     print(
        #         "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
        #             epoch,
        #             batch_id * len(x),
        #             len(loader.dataset),
        #             100.0 * batch_id / len(loader),
        #             loss.item(),
        #         )
        #     )
    